## Tratamento da tabela 'text04'

Neste caderno será apresentada a exploração inicial e tratamento dos dados da tabela 'text04'.

### Importação das bibliotecas
 
A seguir uma breve apresentação de cada biblioteca que será utilizada:

- A biblioteca [NumPy](https://numpy.org/) é fundamental para qualquer tipo de computação científica em Python
- A biblioteca [pandas](https://pandas.pydata.org/) é a nossa ferramenta pricipal para análise e manipulação de dados
- A biblioteca [python-decouple](https://github.com/henriquebastos/python-decouple) auxiliar para trabalhar com variaveis de ambiente
- A biblioteca [SQLAlchemy](https://www.sqlalchemy.org/) é uma biblioteca de mapeamento objeto-relacional SQL

In [1]:
import numpy as np
import pandas as pd

import decouple
import sqlalchemy

### Leitura e tratamento inicial dos dados de entrada

Para poder realizar a conexão com o banco de dados `SQL`, precisaremos de algumas informações que estão armazenadas no arquivo `.env` e quer serão importadas através da biblioteca `decouple`, como a seguir:

In [2]:
user = decouple.config("db_user_mysql")
host = decouple.config('db_host_mysql')
password = decouple.config('db_password_mysql')
database = decouple.config("db_database_mysql")

Através da biblioteca `sqlalchemy` será criado o objeto [Engine](https://docs.sqlalchemy.org/en/14/core/connections.html#sqlalchemy.engine.Engine) que é baseado na URL do banco de dados.

In [ ]:
connection = sqlalchemy.create_engine("mysql+mysqldb://"+user+":"+password+"@"+host+"/"+database)

Será necessário instalar o pacote cujo nome para instalação no Ubuntu 20.04 é `libmysqlclient-dev` para utilizar o driver `mysqldb`, caso não esteja instalado.

Em seguida, através da biblioteca `pandas` é realizada a importação da tabela 'plan02' utilizando a função `read_sql_table()`, onde o parâmetro `con = connection` é o objeto `engine` criado na célula anterior.

In [3]:
df = pd.read_sql_table('text04', con = connection)

,0,1,2,3,4
counter,1,2,3,4,5
paciente,356.0,317.0,188.0,53.0,175.0
nometexto,Pedido exame,Pedido exame,Atestado,Pedido exame,Atestado
texto,<h2>Requisição de exames</h2>\n<br/>\n<p>&nbsp...,<h2>Requisição de exames</h2>\n<br/>\n<p>&nbsp...,<h2>Atestado por Acidente de Trabalho</h2>\n<b...,<h2>Requisição de exames</h2>\n<br/>\n<p>&nbsp...,<h2>Atestado de Sanidade Física e Mental</h2>\...
datatexto,2016-07-03 09:41:03,2016-07-06 18:53:30,2016-07-09 16:31:48,2016-07-12 20:47:14,2016-07-19 06:10:31
timestamp,2021-07-01 18:06:19,2021-07-01 18:06:19,2021-07-01 18:06:19,2021-07-01 18:06:19,2021-07-01 18:06:19
cloud,1,0,0,1,1
update001,2021-05-18 12:51:57,2021-02-14 22:10:44,2021-05-24 20:12:22,2021-04-15 05:15:22,2020-11-24 20:06:14
ch3,1,1,1,1,1
sysuser,5,5,6,2,7


### Visualização dos dados

Nas próximas duas células é realizada a visualização inicial dos dados.

Usando o método `head()` do `pandas` com um argumento `4` nele é possível visualizar os primeiros `4` registros do Dataframe.
    
O `.T` significa `Transposição`, desta forma as linhas serão visualizadas como colunas e vice-versa.

In [ ]:
df.head(4).T

O método `info()` do `pandas` apresenta um resumo dos dados no Dataframe, uma informação interessante é o tipo de dado de cada recurso.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   counter    350 non-null    int64         
 1   paciente   349 non-null    float64       
 2   nometexto  350 non-null    object        
 3   texto      350 non-null    object        
 4   datatexto  350 non-null    datetime64[ns]
 5   timestamp  350 non-null    datetime64[ns]
 6   cloud      350 non-null    int64         
 7   update001  350 non-null    datetime64[ns]
 8   ch3        350 non-null    int64         
 9   sysuser    350 non-null    int64         
dtypes: datetime64[ns](3), float64(1), int64(4), object(2)
memory usage: 27.5+ KB


## Limpeza e tratamento dos dados

A seguir será realizada a limpeza e tratamento dos dados.

### Valores ausentes, valores equivalentes e tratamento inicial

Quando utilizamos o método `info()` para ver o resumo dos dados, foi possível ver que muitas colunas tinham muitos dados ausentes, entrentanto, na documentação da iClinic é possível ver que os campos `patient_birth_date`, `patient_name`, `physician_id` e `date` são obrigatórios e no Dataset anterior as colunas `sysuser` e `datatexto` são seus equivalentes, ainda, nenhuma coluna possui valores nulos nos seus registros, logo, a priori, não será necessário um maior tratamento para cumprir as condições obrigatórias, mas precisaremos obter `patient_birth_date` e `patient_name`.


O tratamento de algumas colunas será realizado em seguida, começando com mudança nos nomes das colunas para seus equivalentes no padrão iClinic.

In [5]:
# Colocando o nome das colunas do arquivo de acordo a documentação da iClinic
df = df.rename(
    columns = {
        "paciente":"patient_id",
        "sysuser": "physician_id",
        "nometexto": "eventprocedure_pack", #tratar depois
        "texto": "description", #tratar depois
        "datatexto": "date",
        "timestamp": "start_time",
        "update001": "end_time",
    }
)

In [6]:
df['status'] = "cp"
df['patient_mobile_phone'] = np.nan
df['arrival_time'] = np.nan #tratar depois
df['all_day'] = np.nan
df['cancel_reason'] = np.nan 
df['event_blocked_scheduling'] = np.nan

#tratar depois
# df['patient_name'] = 
# df['patient_home_phone'] =
# df['patient_email'] =
# df['healthinsurance_name'] =

In [7]:
# Tratamento 'patient_name'
df_patient = pd.read_csv('desafio-base2-output/patient.csv', index_col=["patient_code"], usecols = ["patient_id","name","home_phone","email","healthinsurance_pack"])
df_patient.head(7).T

patient_id,1,2,3,4,5,6,7
name,Nicolas Freitas Teste,Gustavo Silveira Teste,Otávio Barros Teste,Davi Lucas Fogaça Teste,Bruna Santos Teste,Daniel Campos Teste,João Pedro Lima Teste
home_phone,(31)6547-8284,(81)3557-3324,(21)2657-0791,(61)9625-2747,(11)0218-1507,(84)7891-0063,(11)1719-0456
email,luigi31@moura.org,souzanicolas@hotmail.com,tcampos@silva.br,marianaaragao@da.com,ateixeira@barros.net,eloah67@gmail.com,ramosevelyn@alves.org
healthinsurance_pack,"json::[\n\t{\n\t\t""name"":""SulAmérica"",\n\t\t""c...","json::[\n\t{\n\t\t""name"":""ABBESS"",\n\t\t""code""...","json::[\n\t{\n\t\t""name"":""SulAmérica"",\n\t\t""c...",NaN,"json::[\n\t{\n\t\t""name"":""Assistência Médica S...","json::[\n\t{\n\t\t""name"":""Caring Saúde Assistê...","json::[\n\t{\n\t\t""name"":""Allianz"",\n\t\t""code..."


In [8]:
# Tratamento 'patient_name'
df['patient_id'] = df['patient_id'].replace({np.nan: 10000000})
# teste = df['patient_id'].astype(str).str.contains(r'^0')
# print(teste.value_counts())

In [9]:
# Tratamento 'patient_name'
df['patient_id'] = [int(x) if x <= df_patient['name'].size else 0 for x in df['patient_id'] ]
df['patient_name'] = [ df_patient['name'][i] if i != 0 else np.nan for i in df['patient_id'] ]
df['patient_home_phone'] = [ df_patient['home_phone'][i] if i != 0 else np.nan for i in df['patient_id'] ]
df['patient_email'] = [ df_patient['email'][i] if i != 0 else np.nan for i in df['patient_id'] ]

In [10]:
# Removendo a linha que contem NaN pois ter o 'patient_name' é mandatório
# df['patient_name'].value_counts(dropna=False)
df = df.dropna(subset = ['patient_name'])

In [11]:
# Tratamento 'physician_id'
df_physician = pd.read_csv('desafio-base2-output/physician_names.csv', index_col=["sysuser"])
df_physician.head(7).T
# Foi visto que o id varia de 1 - 8

sysuser,1,2,3,4,5,6,7
name,Dra. Ana Paula Lorenti,Dr. Carlos Santana,Dra. Manuela Gonçalves,Dr. Luiz Miguel Pires,Sr. Pietro Cardoso,Sr. Vitor Gabriel Novaes,Dr. Gustavo Porto


In [19]:
# Tratamento 'physician_id'
df['physician_id'] = [ i if i != 0 else np.nan for i in df['physician_id'].astype(int)]
# df['physician_id'].value_counts(dropna=False)
df = df.dropna(subset = ['physician_id'])

In [13]:
# Tratamento 'healthinsurance_name'
df_patient['healthinsurance_pack'] = df_patient['healthinsurance_pack'].str.extract(r'"name":"(.*?)"')[0].values

In [14]:
# Tratamento 'healthinsurance_name'
df['healthinsurance_name'] = [ df_patient['healthinsurance_pack'][i] for i in df['patient_id'] ]

In [15]:
# Exportando a coluna 'eventprocedure_pack' no formato json desejado
df['eventprocedure_pack'] = 'json::[\\n\\t{\\n\\t\\t"name":"'+df['eventprocedure_pack']+'"\\n\\t}\\n]'

In [16]:
df["description"] = '"'+df["description"]+'"'

In [21]:
df = df.dropna(subset = ['date'])

In [17]:
df = df.loc[:, ["patient_id","patient_name","physician_id","date","status","patient_home_phone","patient_mobile_phone","arrival_time","start_time","end_time","description","all_day","cancel_reason","patient_email","event_blocked_scheduling","healthinsurance_name","eventprocedure_pack"]]

In [22]:
df.to_csv('desafio-base2-output/event_scheduling.csv',index=False, encoding='utf-8')